# Malign

`malign` is a set of experimental methods for sequence alignment that allow:
    
    - a unique alphabet for each sequence
    - computation of k-best alignments
    - usage of assymetric information
    - perform single-pass true multiple alignment (instead of combination of pairwise)
    
This notebook explores ideas on how to use it.

In [1]:
# Import the library
import malign
from malign import print_alms, print_malms

The most stupid alignment method implemented in `malign` is the "dumb" one. It returns a single alignment for a number of sequences, padding spaces to the left and right without even considering the values of the sequences. It is used for bootstrapping a number of other alignment methods.

In [2]:
# Perform pairwise dumb alignment
seq_a = 'tra'
seq_b = 'fatata'
print_alms(malign.pw_align(seq_a, seq_b, method="dumb"))

# Perform multiwise dumb alignment
seqs = ['tra', 'fra', 'batata', 'virp', 'x']
print_malms(malign.multi_align(seqs, method="dumb"))

A 0 (0 / 0): - t r a - -
B 0 (0 / 0): f a t a t a
0 A {'seq': ['-', 't', 'r', 'a', '-', '-'], 'score': 0.5}
0 B {'seq': ['-', 'f', 'r', 'a', '-', '-'], 'score': 0.5}
0 C {'seq': ['b', 'a', 't', 'a', 't', 'a'], 'score': 1.0}
0 D {'seq': ['-', 'v', 'i', 'r', 'p', '-'], 'score': 0.6666666666666667}
0 E {'seq': ['-', '-', 'x', '-', '-', '-'], 'score': 0.16666666666666663}
